# Sea Surface Temperature

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import scipy.stats as stats
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect
from regions import boolean_mask, SOM_area, Nino12, Nino34, global_ocean,\
                    gl_ocean_rect, NPacific_mask_rect,\
                    Nino12_low, Nino34_low
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression

# SST indices

In [ ]:
MASK_ocn  = boolean_mask('ocn'     , 0)
MASK_low  = boolean_mask('ocn_low' , 0)
MASK_rect = boolean_mask('ocn_rect', 0)

In [ ]:
TAREA       = xr_AREA('ocn')
AREA_low    = xr_AREA('ocn_low')
AREA_rect   = xr_AREA('ocn_rect')

In [ ]:
AREA_low.plot()

In [ ]:
global_area = TAREA.where(MASK_ocn).sel(global_ocean).sum()
print(f'{global_area.item():e}')
global_area2 = AREA_rect.where(MASK_rect).sel(gl_ocean_rect).sum()
print(f'{global_area2.item():e}')

## ENSO

In [ ]:
# %%time
# #5-10 sec per year
# Nino12_AREA = TAREA.sel(Nino12).sum(dim=('nlat','nlon'))
# Nino34_AREA = TAREA.sel(Nino34).sum(dim=('nlat','nlon'))
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='monthly')):
#         if m==1: print(y)
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,0,:,:]
#         N12 = SST_index(xa_SST=SST, AREA=TAREA, index_loc=Nino12, AREA_index=Nino12_AREA, MASK=MASK_ocn, dims=('nlat', 'nlon'))
#         N34 = SST_index(xa_SST=SST, AREA=TAREA, index_loc=Nino34, AREA_index=Nino34_AREA, MASK=MASK_ocn, dims=('nlat', 'nlon'))
#         if i==0:
#             N12_new = N12
#             N34_new = N34
#         else:
#             N12_new = xr.concat([N12_new, N12], dim='time')
#             N34_new = xr.concat([N34_new, N34], dim='time')
#     N12_new.to_netcdf(path=f'{path_samoc}/SST/Nino12_raw_{run}.nc')
#     N34_new.to_netcdf(path=f'{path_samoc}/SST/Nino34_raw_{run}.nc')

In [ ]:
# %%time
# # 1:04 hrs for both lpi, lpd
# Nino12_low_AREA = AREA_low.sel(Nino12_low).sum(dim=('nlat','nlon'))
# Nino34_low_AREA = AREA_low.sel(Nino34_low).sum(dim=('nlat','nlon'))
# for run in ['lpi', 'lpd']:# ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='monthly')):
#         if m==1: print(y)
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,0,:,:]
#         N12 = SST_index(xa_SST=SST, AREA=AREA_low, index_loc=Nino12_low, AREA_index=Nino12_low_AREA, MASK=MASK_low, dims=('nlat', 'nlon'))
#         N34 = SST_index(xa_SST=SST, AREA=AREA_low, index_loc=Nino34_low, AREA_index=Nino34_low_AREA, MASK=MASK_low, dims=('nlat', 'nlon'))
#         if i==0:
#             N12_new = N12
#             N34_new = N34
#         else:
#             N12_new = xr.concat([N12_new, N12], dim='time')
#             N34_new = xr.concat([N34_new, N34], dim='time')
#     N12_new.to_netcdf(path=f'{path_samoc}/SST/Nino12_raw_{run}.nc')
#     N34_new.to_netcdf(path=f'{path_samoc}/SST/Nino34_raw_{run}.nc')

In [ ]:
N12_ctrl = xr.open_dataarray(f'{path_samoc}/SST/Nino12_raw_ctrl.nc', decode_times=False)
N34_ctrl = xr.open_dataarray(f'{path_samoc}/SST/Nino34_raw_ctrl.nc', decode_times=False)
N12_rcp  = xr.open_dataarray(f'{path_samoc}/SST/Nino12_raw_rcp.nc' , decode_times=False)
N34_rcp  = xr.open_dataarray(f'{path_samoc}/SST/Nino34_raw_rcp.nc' , decode_times=False)
N12_lpd  = xr.open_dataarray(f'{path_samoc}/SST/Nino12_raw_lpd.nc' , decode_times=False)
N34_lpd  = xr.open_dataarray(f'{path_samoc}/SST/Nino34_raw_lpd.nc' , decode_times=False)
N12_lpi  = xr.open_dataarray(f'{path_samoc}/SST/Nino12_raw_lpi.nc' , decode_times=False)
N34_lpi  = xr.open_dataarray(f'{path_samoc}/SST/Nino34_raw_lpi.nc' , decode_times=False)

In [ ]:
# linear trend
def linear_fit(ts):
    lp = np.polyfit(ts.time, ts.values, 1)
    lf = lp[0]*ts.time + lp[1]
    return lf

N12_lf_ctrl = linear_fit(N12_ctrl)
N34_lf_ctrl = linear_fit(N34_ctrl)
N12_lf_rcp  = linear_fit(N12_rcp)
N34_lf_rcp  = linear_fit(N34_rcp)
N12_lf_lpd  = linear_fit(N12_lpd)
N34_lf_lpd  = linear_fit(N34_lpd)
N12_lf_lpi  = linear_fit(N12_lpi)
N34_lf_lpi  = linear_fit(N34_lpi)

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
# ax.axhline(0,c='k', lw=.5)

ax.plot(N34_ctrl.time/365+1900, N34_ctrl, c='C0', ls=':', lw=.15)
ax.plot(N34_rcp .time/365+ 200, N34_rcp , c='C1', ls=':', lw=.15)
ax.plot(N34_lpd .time/365+1400, N34_lpd , c='C2', ls=':', lw=.15)
ax.plot(N34_lpi .time/365-1600, N34_lpi , c='C3', ls=':', lw=.15)

ax.plot(N12_ctrl.time/365+1900, N12_ctrl, c='C0', ls=':', lw=.15)
ax.plot(N12_rcp .time/365+ 200, N12_rcp , c='C1', ls=':', lw=.15)
ax.plot(N12_lpd .time/365+1400, N12_lpd , c='C2', ls=':', lw=.15)
ax.plot(N12_lpi .time/365-1600, N12_lpi , c='C3', ls=':', lw=.15)

ax.plot(N34_ctrl.time/365+1900, N34_lf_ctrl, c='C0', ls='--', lw=2)
ax.plot(N34_rcp .time/365+ 200, N34_lf_rcp , c='C1', ls='--', lw=2)
ax.plot(N34_lpd .time/365+1400, N34_lf_lpd , c='C2', ls='--', lw=2)
ax.plot(N34_lpi .time/365-1600, N34_lf_lpi , c='C3', ls='--', lw=2)

ax.plot(N12_ctrl.time/365+1900, N12_lf_ctrl, c='C0', ls='--', lw=2)
ax.plot(N12_rcp .time/365+ 200, N12_lf_rcp , c='C1', ls='--', lw=2)
ax.plot(N12_lpd .time/365+1400, N12_lf_lpd , c='C2', ls='--', lw=2)
ax.plot(N12_lpi .time/365-1600, N12_lf_lpi , c='C3', ls='--', lw=2)

# ax.plot(N34_ctrl.time/365+1900, lowpass(N34_ctrl.values, 40), c='C0', ls='-' , lw=2)
# ax.plot(N34_rcp .time/365+ 200, lowpass(N34_rcp .values, 40), c='C1', ls='-' , lw=2)
# ax.plot(N34_lpd .time/365+1400, lowpass(N34_lpd .values, 40), c='C2', ls='-' , lw=2)
# ax.plot(N34_lpi .time/365-1600, lowpass(N34_lpi .values, 40), c='C3', ls='-' , lw=2)

# ax.plot(N12_ctrl.time/365+1900, lowpass(N12_ctrl.values, 100), c='C0', ls='-' , lw=2)
# ax.plot(N12_rcp .time/365+ 200, lowpass(N12_rcp .values, 100), c='C1', ls='-' , lw=2)
# ax.plot(N12_lpd .time/365+1400, lowpass(N12_lpd .values, 100), c='C2', ls='-' , lw=2)
# ax.plot(N12_lpi .time/365-1600, lowpass(N12_lpi .values, 100), c='C3', ls='-' , lw=2)

ax.text(   0, 30.5, 'pre-ind. low' , fontsize=16, color='C3')
ax.text(1550, 30.5, 'pres. day low', fontsize=16, color='C2')
ax.text(2000, 30.5, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, 30.5, 'RCP'          , fontsize=16, color='C1')

ax.set_xlabel('time[years]', fontsize=16)
ax.set_ylabel('N12 / N34 index [$^\circ$C]'  , fontsize=16)
f.tight_layout()
plt.savefig(f'{path_results}/SST/ENSO_raw')

In [ ]:
plt.figure(figsize=(12,5))
plt.tick_params(labelsize=14)

plt.plot(N12_ctrl.time/365, N12_ctrl, c='C0', ls='-' ,  alpha=.5, label='CTRL Nino12')
plt.plot(N12_ctrl.time/365, N12_lf_ctrl   , c='C0', ls='--', lw=2 )
plt.plot(N34_ctrl.time/365, N34_ctrl, c='C0', ls='-' ,  alpha=.5, label='CTRL Nino34')
plt.plot(N12_ctrl.time/365, N34_lf_ctrl   , c='C0', ls='--', lw=2 )
plt.axvline(200, c='g', lw=.5)
plt.plot(N12_rcp.time/365-1700, N12_rcp, c='C1', ls='-' , alpha=.5, label='RCP Nino12')
plt.plot(N12_rcp.time/365-1700, N12_lf_rcp   , c='C1', ls='--', lw=2 )
plt.plot(N34_rcp.time/365-1700, N34_rcp, c='C1', ls='-' , alpha=.5, label='RCP Nino34')
plt.plot(N34_rcp.time/365-1700, N34_lf_rcp   , c='C1', ls='--', lw=2 )
plt.ylabel('monthly temperature [$^\circ$C]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.tight_layout()
# plt.legend(handles=[L2, L1, L4, L3], ncol=2, fontsize=14, loc=9, frameon=False)
# plt.savefig(f'{path_results}/SST/ENSO_raw')

In [ ]:
def seasonal_mean_std(ts):
    mean = np.empty((12))
    std  = np.empty((12))
    for i in range(12):
        mean[i] = (ts - linear_fit(ts) )[i::12].mean()
        std[i]  = (ts - linear_fit(ts) )[i::12].std()
    return mean, std

# seasonal cycle
N12_mean_ctrl, N12_std_ctrl = seasonal_mean_std(N12_ctrl)
N34_mean_ctrl, N34_std_ctrl = seasonal_mean_std(N34_ctrl)
N12_mean_rcp , N12_std_rcp  = seasonal_mean_std(N12_rcp)
N34_mean_rcp , N34_std_rcp  = seasonal_mean_std(N34_rcp)
N12_mean_lpd , N12_std_lpd  = seasonal_mean_std(N12_lpd)
N34_mean_lpd , N34_std_lpd  = seasonal_mean_std(N34_lpd)
N12_mean_lpi , N12_std_lpi  = seasonal_mean_std(N12_lpi)
N34_mean_lpi , N34_std_lpi  = seasonal_mean_std(N34_lpi)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12,5), sharey=True)
RD = mlines.Line2D([], [], color='C3', marker='D', linestyle='None', markersize=5, label='LPI')
GD = mlines.Line2D([], [], color='C2', marker='D', linestyle='None', markersize=5, label='LPD')
BD = mlines.Line2D([], [], color='C0', marker='o', linestyle='None', markersize=5, label='CTRL')
YD = mlines.Line2D([], [], color='C1', marker='o', linestyle='None', markersize=5, label='RCP')

for i in range(2):
    ax[i].tick_params(labelsize=14)
    ax[i].set_xlabel('month', fontsize=16)
    ax[i].set_xticks(np.arange(1,13))
    ax[i].axhline(0, c='k', lw=.5)
    ax[i].legend(handles=[RD, GD, BD, YD], ncol=2, fontsize=14, frameon=False)

for i in range(12):
    ax[0].scatter([i+1.1], N12_mean_ctrl[i], c='C0', label='N12 CTRL')
    ax[0].scatter([i+1.3], N12_mean_rcp [i], c='C1', label='N12 RCP')
    ax[0].scatter([i+0.9], N12_mean_lpd [i], c='C2', marker='D', label='N12 LPD')
    ax[0].scatter([i+0.7], N12_mean_lpi [i], c='C3', marker='D', label='N12 LPI')

    ax[0].plot(2* [i+1.1], [N12_mean_ctrl[i]-N12_std_ctrl[i], N12_mean_ctrl[i]+N12_std_ctrl[i]], c='C0')
    ax[0].plot(2* [i+1.3], [N12_mean_rcp [i]-N12_std_rcp[i] , N12_mean_rcp[i] +N12_std_rcp[i]], c='C1')
    ax[0].plot(2* [i+0.9], [N12_mean_lpd [i]-N12_std_lpd[i] , N12_mean_lpd[i] +N12_std_lpd[i]], c='C2')
    ax[0].plot(2* [i+0.7], [N12_mean_lpi [i]-N12_std_lpi[i] , N12_mean_lpi[i] +N12_std_lpi[i]], c='C3')

    ax[1].scatter([i+1.1], N34_mean_ctrl[i], c='C0', label='N34 CTRL')
    ax[1].scatter([i+1.3], N34_mean_rcp [i], c='C1', label='N34 RCP')
    ax[1].scatter([i+0.9], N34_mean_lpd [i], c='C2', marker='D',label='N34 LPD')
    ax[1].scatter([i+0.7], N34_mean_lpi [i], c='C3', marker='D',label='N34 LPI')

    ax[1].plot(2* [i+1.1], [N34_mean_ctrl[i]-N34_std_ctrl[i], N34_mean_ctrl[i]+N34_std_ctrl[i]], c='C0')
    ax[1].plot(2* [i+1.3], [N34_mean_rcp [i]-N34_std_rcp[i] , N34_mean_rcp[i] +N34_std_rcp[i] ], c='C1')
    ax[1].plot(2* [i+0.9], [N34_mean_lpd [i]-N34_std_lpd[i] , N34_mean_lpd[i] +N34_std_lpd[i] ], c='C2')
    ax[1].plot(2* [i+0.7], [N34_mean_lpi [i]-N34_std_lpi[i] , N34_mean_lpi[i] +N34_std_lpi[i] ], c='C3')

ax[0].text(.5,2.7,'N12', fontsize=16)
ax[1].text(.5,2.7,'N34', fontsize=16)
    
ax[0].set_ylabel('temperature anomaly [K]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/SST/ENSO_seasonal_cycle')

Claudia suggested to remove the seasonal signal (averages of all the Januaries, Februaries, ...). First I will remove a linear trend from both the CTRL and RCP indices.

In [ ]:
# detrend, deseasonalize
def detrend_deseason(ts):
    dtds = ts - linear_fit(ts)
    seas = seasonal_mean_std(ts)[0]
    for i in range(12):
        dtds[i::12] = dtds[i::12] - seas[i]
    return dtds

In [ ]:
detrend_deseason(N12_ctrl).to_netcdf(f'{path_results}/SST/Nino12_ctrl.nc')
detrend_deseason(N12_rcp ).to_netcdf(f'{path_results}/SST/Nino12_rcp.nc' )
detrend_deseason(N12_lpd ).to_netcdf(f'{path_results}/SST/Nino12_lpd.nc' )
detrend_deseason(N12_lpi ).to_netcdf(f'{path_results}/SST/Nino12_lpi.nc' )

detrend_deseason(N34_ctrl).to_netcdf(f'{path_results}/SST/Nino34_ctrl.nc')
detrend_deseason(N34_rcp ).to_netcdf(f'{path_results}/SST/Nino34_rcp.nc' )
detrend_deseason(N34_lpd ).to_netcdf(f'{path_results}/SST/Nino34_lpd.nc' )
detrend_deseason(N34_lpi ).to_netcdf(f'{path_results}/SST/Nino34_lpi.nc' )

In [ ]:
N12_dtds_ctrl = xr.open_dataarray(f'{path_results}/SST/Nino12_ctrl.nc', decode_times=False)
N12_dtds_rcp  = xr.open_dataarray(f'{path_results}/SST/Nino12_rcp.nc' , decode_times=False)
N12_dtds_lpd  = xr.open_dataarray(f'{path_results}/SST/Nino12_lpd.nc' , decode_times=False)
N12_dtds_lpi  = xr.open_dataarray(f'{path_results}/SST/Nino12_lpi.nc' , decode_times=False)

N34_dtds_ctrl = xr.open_dataarray(f'{path_results}/SST/Nino34_ctrl.nc', decode_times=False)
N34_dtds_rcp  = xr.open_dataarray(f'{path_results}/SST/Nino34_rcp.nc' , decode_times=False)
N34_dtds_lpd  = xr.open_dataarray(f'{path_results}/SST/Nino34_lpd.nc' , decode_times=False)
N34_dtds_lpi  = xr.open_dataarray(f'{path_results}/SST/Nino34_lpi.nc' , decode_times=False)

In [ ]:
x = np.arange(-3.5,4.25,.05)
N12s = [N12_ctrl, N12_rcp, N12_lpd, N12_lpi]
N34s = [N34_ctrl, N34_rcp, N34_lpd, N34_lpi]

f, ax = plt.subplots(1, 2, figsize=(12,5), sharey=True)
ax[0].set_ylabel('probability density', fontsize=16)
ax[0].text(-3.5, .7, 'Nino12', fontsize=16)
ax[1].text(-3.5, .7, 'Nino34', fontsize=16)

for i in range (4):
    label= ['CTRL', 'RCP', 'LPD', 'LPI'][i]
    density = stats.gaussian_kde(N12s[i])
    ax[0].plot(x, density(x), label=label)
    density = stats.gaussian_kde(N34s[i])
    ax[1].plot(x, density(x), label=label)

for i in range(2):
    ax[i].axhline(0, c='k', lw=.5)
    ax[i].axvline(0, c='k', lw=.5)
    ax[i].tick_params(labelsize=14)
    ax[i].set_xlabel('index anomaly [K]', fontsize=16)
    ax[i].legend(fontsize=14, frameon=False)
    
plt.tight_layout()
plt.savefig(f'{path_results}/SST/ENSO_kde')

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(12,8), sharex=True)
for i in range(2):
    ax[i].tick_params(labelsize=14)

ax[0].axhline(0, c='k', lw=.5)
ax[0].axvline(200, c='g', lw=.5)
ax[0].axvline(300, c='g', lw=.5)
ax[0].plot(N12_dtds_ctrl.time/365    , N12_dtds_ctrl.rolling(time=6  , center=True).mean(), c='C0')
ax[0].plot(N12_dtds_ctrl.time/365    , N12_dtds_ctrl.rolling(time=240, center=True).std() , c='C0', ls=':', lw=2)
ax[0].plot(N12_dtds_rcp.time/365-1700, N12_dtds_rcp .rolling(time=6  , center=True).mean(), c='C1')
ax[0].plot(N12_dtds_rcp.time/365-1700, N12_dtds_rcp .rolling(time=240, center=True).std() , c='C1', ls=':', lw=2)

ax[1].axhline(0, c='k', lw=.5)
ax[1].axvline(200, c='g', lw=.5)
ax[1].axvline(300, c='g', lw=.5)
ax[1].plot(N34_dtds_ctrl.time/365    , N34_dtds_ctrl.rolling(time=6  , center=True).mean(), c='C0')
ax[1].plot(N34_dtds_ctrl.time/365    , N34_dtds_ctrl.rolling(time=240, center=True).std() , c='C0', ls=':', lw=2)
ax[1].plot(N34_dtds_rcp.time/365-1700, N34_dtds_rcp .rolling(time=6  , center=True).mean(), c='C1')
ax[1].plot(N34_dtds_rcp.time/365-1700, N34_dtds_rcp .rolling(time=240, center=True).std() , c='C1', ls=':', lw=2)
ax[1].set_xticks(np.arange(100,400,20))

ax[1].set_xlabel('time [years]'     , fontsize=16)
ax[0].set_ylabel('Nino1+2 index [K]', fontsize=16)
ax[1].set_ylabel('Nino3.4 index [K]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/SST/ENSO_indices_rstd')

In [ ]:
# FFT
f, t, Sxx = sp.signal.spectrogram(N34_lpi, fs=12)
plt.pcolormesh(t, f, Sxx)
plt.ylim((0,.6))

In [ ]:
N12s = [N12_dtds_ctrl, N12_dtds_rcp, N12_dtds_lpd, N12_dtds_lpi]
N34s = [N34_dtds_ctrl, N34_dtds_rcp, N34_dtds_lpd, N34_dtds_lpi]

f, ax = plt.subplots(1, 2, figsize=(12,5), sharey=True)
ax[0].set_ylabel('spectral power', fontsize=16)

for i in range (4):
    label= ['CTRL', 'RCP', 'LPD', 'LPI'][i]
    f, Pxx = sp.signal.welch(N12s[i], fs=12)
    ax[0].semilogy(1/f, np.sqrt(Pxx), label=label)
    f, Pxx = sp.signal.welch(N34s[i], fs=12)
    ax[1].semilogy(1/f, np.sqrt(Pxx), label=label)

for i in range(2):
    ax[i].tick_params(labelsize=14)
    ax[i].set_xlabel('period [yr]', fontsize=16)
    ax[i].legend(fontsize=14, frameon=False)
    
plt.tight_layout()
plt.savefig(f'{path_results}/SST/ENSO_spectrum')

In [ ]:
# correlation between indices
plt.figure()
plt.axhline(0, c='k', lw=.5)
plt.axvline(0, c='k', lw=.5)
plt.scatter(N12_detr_des_ctrl, N34_detr_des_ctrl)
plt.scatter(N12_detr_des_rcp , N34_detr_des_rcp )

# lagged correlation
plt.figure()
plt.axhline(0, c='k', lw=.5)
plt.axvline(0, c='k', lw=.5)
for l in np.arange(-15,16,1):
    if l<0:
        plt.scatter(l, np.corrcoef(N12_detr_des_ctrl[:l], N34_detr_des_ctrl.shift(time=l).dropna(dim='time'))[0,1], c='C0')
        plt.scatter(l, np.corrcoef(N12_detr_des_rcp [:l], N34_detr_des_rcp .shift(time=l).dropna(dim='time'))[0,1], c='C1', marker='D')
    else:
        plt.scatter(l, np.corrcoef(N12_detr_des_ctrl[l:], N34_detr_des_ctrl.shift(time=l).dropna(dim='time'))[0,1], c='C0')
        plt.scatter(l, np.corrcoef(N12_detr_des_rcp [l:], N34_detr_des_rcp .shift(time=l).dropna(dim='time'))[0,1], c='C1', marker='D')